In [16]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json

#from lxml import html

In [17]:
#Here is a list of URLS, please update with current
ListOfURL = ['http://www.gradesource.com/reports/288/29963/coursestand.html', 'http://www.gradesource.com/reports/7/29889/cs126276.html', 'http://www.gradesource.com/reports/7/29887/coursestand.html', 'http://www.gradesource.com/reports/430/29962/coursestand.html']
ListOfCourse = ['CSE30_Sp18', 'CSE110_Sp18', 'CSE12_Sp18', 'CSE7_Sp18']

In [18]:
def getGradeSourceGrades(URL):
    arrTotal = []

    quote_page = URL
    #URL of gradesource website that needs to be scraped
    page = requests.get(quote_page)
    #making a request
    soup = BeautifulSoup(page.content, 'html.parser')
    #cleaning the html
    
    table = soup.find_all('tr')
    #retrieving all the table
    arr = []

    for i in range(len(table)):
        arr.append(table[i].find_all('td'))

    arrHolder = {}
    for i in range(len(arr)):
        if i >= 8 and i < len(arr):     
            if arr[i][0].find("b") != None:

                arrHolder[arr[i][0].find("b").text] = {}
                
                if arr[i][len(arr[i])-1].find("b") != None:
               
                    arrHolder[arr[i][0].find("b").text]["Grade"] = arr[i][len(arr[i])-1].find("b").text  # getting the course id
              
                elif arr[i][len(arr[i])-1].find("font") != None:
                
                    arrHolder[arr[i][0].find("b").text]["Grade"] = arr[i][len(arr[i])-1].find("font").text # getting the rank
           
                arrHolder[arr[i][0].find("b").text]["Rank"] = arr[i][len(arr[i])-2].find("font").text + "/" + str(len(arr)-4)
            elif arr[i][0].find("font") != None:

                arrHolder[arr[i][0].find("font").text] = {}
                
                if arr[i][len(arr[i])-1].find("b") != None:

                    arrHolder[arr[i][0].find("font").text]["Grade"] = arr[i][len(arr[i])-1].find("b").text # getting the course id

                elif arr[i][len(arr[i])-1].find("font") != None:

                    arrHolder[arr[i][0].find("font").text]["Grade"] = arr[i][len(arr[i])-1].find("font").text # getting the rank
                
                arrHolder[arr[i][0].find("font")]["Rank"] = str(arr[i][len(arr[i])-2].find("font").text) + "/" + str(len(arr)-4)
    #arrTotal.append(arrHolder)
    return arrHolder

In [19]:
#this code is used for formatting into a json 
for i in range(len(ListOfURL)):
    from firebase import firebase
    CSECourses = getGradeSourceGrades(ListOfURL[i])


    courseList = json.dumps(CSECourses, indent=4)
    courseList = courseList.replace("/", "／")
    courseList = courseList.replace(".", "．")

    jsonDict = json.loads(courseList)

    #print(courseList)
    jsonDict = json.loads(courseList)
    firebase = firebase.FirebaseApplication('https://csehub-420.firebaseio.com/')

    #This code is intended to send the dictionary to firebase
    result = firebase.patch('/GradeSource', { ListOfCourse[i]: jsonDict})